<a href="https://colab.research.google.com/github/sheldonkemper/bank_of_england/blob/tidy_up_preprocessing_notebook/notebooks/processed/ct_preprocessing_jpmorgan_gp4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [228]:
"""
===================================================
Author: Chiaki Tachikawa
Role: Data Science Lead, Bank of England Employer Project (Quant Collective)
LinkedIn: https://www.linkedin.com/in/chiaki-tachikawa
Date: 2025-02-13
Version: 1.1

Description:
    This notebook implements a system for cleaning and exporting transcript data for the Bank of England project. The workflow includes:
    - Importing necessary libraries and downloading NLTK data.
    - Defining and applying a `preprocessor` function to clean and tokenize text data.
    - Reading and preprocessing various CSV files containing transcript data.
    - Exporting the preprocessed data to new CSV files for further analysis.

===================================================
"""

'\n===================================================\nAuthor: Chiaki Tachikawa\nRole: Data Science Lead, Bank of England Employer Project (Quant Collective)\nLinkedIn: https://www.linkedin.com/in/chiaki-tachikawa\nDate: 2025-02-13\nVersion: 1.1\n\nDescription:\n    This notebook implements a system for cleaning and exporting transcript data for the Bank of England project. The workflow includes:\n    - Importing necessary libraries and downloading NLTK data.\n    - Defining and applying a `preprocessor` function to clean and tokenize text data.\n    - Reading and preprocessing various CSV files containing transcript data.\n    - Exporting the preprocessed data to new CSV files for further analysis.\n\n===================================================\n'

# **Library**

In [229]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download("stopwords")
nltk.download("punkt")
nltk.download("punkt_tab")
nltk.download('wordnet')
from nltk.tokenize import word_tokenize, sent_tokenize
from collections import Counter
import regex as re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from google.colab import drive


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


# **Function**

preprocessor function : The function modifies the DataFrame data in place, adding two new columns (col1 and col2) with preprocessed text.


Input:
  - name of dataframe
  - name of column which contains the text to clean
  - name of column which is tokenized
  - name of column which is cleaned

In [230]:
#create function to preprocess data
def preprocessor (data, col, col1,col2):
  #Copy col1umn
  data[col1]=data[col]
  data[col2]=data[col]


  #Adding column1
  #Lower the lettercase
  data[col1] = data[col1].str.lower()

  #Remove stop words
  stop_words = set(stopwords.words("english"))
  data[col1] = data[col1].apply(lambda x: " ".join([word for word in str(x).split() if word not in (stop_words)]))

  #Tokenize the word
  data[col1] = data[col1].apply(nltk.word_tokenize)

  #Remove numbers
  data[col1] = data[col1].apply(lambda x: [word for word in x if not word.isdigit()])

  #remove symbol from comments
  data[col1] = data[col1].apply(lambda x: [word for word in x if x!=""])

  #remove short word
  data[col1] = data[col1].apply(lambda x: [word for word in x if len(word)>2])

  #remove symbols
  data[col1] = data[col1].apply (lambda x: [re.sub(r"[^a-z]", "", word) for word in x])

  #lemmatization
  lemmatizer = WordNetLemmatizer()
  data[col1] = data[col1].apply(lambda x: [lemmatizer.lemmatize(word) for word in x])



  #Adding column2
  #Lower the lettercase
  data[col2] = data[col2].str.lower()

  #Remove stop words
  stop_words = set(stopwords.words("english"))
  data[col2] = data[col2].apply(lambda x: " ".join([word for word in str(x).split() if word not in (stop_words)]))

  #remove symbols
  data[col2] = data[col2].apply (lambda x: [re.sub(r"[.,'?]", "", x)])

  return


find_row: This function searches upwards from the given current_row_num in the DataFrame df to find the first row where the value in column "M" is "A". It returns the index of that row. If no such row is found, it returns 0

In [231]:
def find_row (df, col, current_row_num):
  #list_name=[]
  i = current_row_num-1
  while i > 0:
    if df[col][i] == "A":
      break
    else:
      i-=1
  return i

find_row_empty: This function searches upwards from the given current_row_num to find the first row where col1 has the value "A" and col2 is not an empty list. It returns the index of that row.

## **Data**

In [232]:
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


JP Morgan QA section

In [233]:
#Defining qa_data
qa_data = pd.read_excel("/content/drive/MyDrive/bank_of_england/data/preprocessed_data/JP Mogran processed thru OpenAI/JPMorgan_QNA_Consildated_processed_data.xlsx")
qa_data.head()

,Index,Quarter-Year,Question,Asked By,Role of the person asked the question,Answer,Answered By,Role of the person answered the question
0,1,1Q23,"So, Jamie, I was actually hoping to get your p...",Steven Chubak,"Analyst, Wolfe Research LLC","Well, I think you were already kind of complet...",Jamie Dimon,"Chairman & Chief Executive Officer, JPMorgan C..."
1,2,1Q23,"Hey, thanks. Good morning. Hey, Jeremy, I was ...",Ken Usdin,"Analyst, Jefferies LLC","Yeah, sure. So let me just summarize the drive...",Jeremy Barnum,"Chief Financial Officer, JPMorgan Chase & Co."
2,3,1Q23,"Hi, thanks. Jeremy, wanted to follow up again ...",John McDonald,"Analyst, Autonomous Research","Yeah. John, it's a really good question, and w...",Jeremy Barnum,"Chief Financial Officer, JPMorgan Chase & Co."
3,4,1Q23,My first question is you mentioned that your r...,Erika Najarian,"Analyst, UBS Securities LLC","Yeah. So, Erika, as you know, we take \n not g...",Jeremy Barnum,"Chief Financial Officer, JPMorgan Chase & Co."
4,5,1Q23,Hey. Good morning. Maybe just a little bit on ...,Jim Mitchell,"Analyst, Seaport Global Securities LLC","Yeah. A couple things there. So, first of all,...","Jeremy Barnum, Jamie Dimon","Chief Financial Officer, JPMorgan Chase & Co.;..."


In [234]:
#preprocessing data
preprocessor(qa_data, "Question", "question_tokenised_data", "question_cleaned")
preprocessor(qa_data,"Answer","answer_tokenised_data","answer_cleaned")

In [235]:
#remove operater
#qa_data = qa_data.loc[qa_data["speaker"]!="Operator"]

#remove less than 20 words
qa_data["count"] = qa_data["question_tokenised_data"].apply(lambda x: len(x))


In [236]:
#qa_data.to_csv("qa_data.csv")

In [237]:
qa_data = qa_data.loc[qa_data["count"]>5]
qa_data.head()

,Index,Quarter-Year,Question,Asked By,Role of the person asked the question,Answer,Answered By,Role of the person answered the question,question_tokenised_data,question_cleaned,answer_tokenised_data,answer_cleaned,count
0,1,1Q23,"So, Jamie, I was actually hoping to get your p...",Steven Chubak,"Analyst, Wolfe Research LLC","Well, I think you were already kind of complet...",Jamie Dimon,"Chairman & Chief Executive Officer, JPMorgan C...","[jamie, actually, hoping, get, perspective, se...",[so jamie actually hoping get perspective see ...,"[well, think, already, kind, complete, answeri...",[well think already kind complete answering qu...,54
1,2,1Q23,"Hey, thanks. Good morning. Hey, Jeremy, I was ...",Ken Usdin,"Analyst, Jefferies LLC","Yeah, sure. So let me just summarize the drive...",Jeremy Barnum,"Chief Financial Officer, JPMorgan Chase & Co.","[hey, thanks, good, morning, hey, jeremy, wond...",[hey thanks good morning hey jeremy wondering ...,"[yeah, sure, let, summarize, driver, change, o...",[yeah sure let summarize drivers change outloo...,35
2,3,1Q23,"Hi, thanks. Jeremy, wanted to follow up again ...",John McDonald,"Analyst, Autonomous Research","Yeah. John, it's a really good question, and w...",Jeremy Barnum,"Chief Financial Officer, JPMorgan Chase & Co.","[thanks, jeremy, wanted, follow, driver, nii, ...",[hi thanks jeremy wanted follow drivers nii re...,"[yeah, john, really, good, question, obviously...",[yeah john really good question obviously thou...,45
3,4,1Q23,My first question is you mentioned that your r...,Erika Najarian,"Analyst, UBS Securities LLC","Yeah. So, Erika, as you know, we take \n not g...",Jeremy Barnum,"Chief Financial Officer, JPMorgan Chase & Co.","[first, question, mentioned, reserve, build, d...",[first question mentioned reserve build driven...,"[yeah, erika, know, take, going, lot, detail, ...",[yeah so erika know take going go lot detail t...,17
4,5,1Q23,Hey. Good morning. Maybe just a little bit on ...,Jim Mitchell,"Analyst, Seaport Global Securities LLC","Yeah. A couple things there. So, first of all,...","Jeremy Barnum, Jamie Dimon","Chief Financial Officer, JPMorgan Chase & Co.;...","[hey, good, morning, maybe, little, bit, depos...",[hey good morning maybe little bit deposit tho...,"[yeah, couple, thing, there, first, all, know,...",[yeah couple things there so first all know ri...,31


In [238]:
#reset index
qa_data.reset_index(drop=True, inplace=True)
qa_data.head()

,Index,Quarter-Year,Question,Asked By,Role of the person asked the question,Answer,Answered By,Role of the person answered the question,question_tokenised_data,question_cleaned,answer_tokenised_data,answer_cleaned,count
0,1,1Q23,"So, Jamie, I was actually hoping to get your p...",Steven Chubak,"Analyst, Wolfe Research LLC","Well, I think you were already kind of complet...",Jamie Dimon,"Chairman & Chief Executive Officer, JPMorgan C...","[jamie, actually, hoping, get, perspective, se...",[so jamie actually hoping get perspective see ...,"[well, think, already, kind, complete, answeri...",[well think already kind complete answering qu...,54
1,2,1Q23,"Hey, thanks. Good morning. Hey, Jeremy, I was ...",Ken Usdin,"Analyst, Jefferies LLC","Yeah, sure. So let me just summarize the drive...",Jeremy Barnum,"Chief Financial Officer, JPMorgan Chase & Co.","[hey, thanks, good, morning, hey, jeremy, wond...",[hey thanks good morning hey jeremy wondering ...,"[yeah, sure, let, summarize, driver, change, o...",[yeah sure let summarize drivers change outloo...,35
2,3,1Q23,"Hi, thanks. Jeremy, wanted to follow up again ...",John McDonald,"Analyst, Autonomous Research","Yeah. John, it's a really good question, and w...",Jeremy Barnum,"Chief Financial Officer, JPMorgan Chase & Co.","[thanks, jeremy, wanted, follow, driver, nii, ...",[hi thanks jeremy wanted follow drivers nii re...,"[yeah, john, really, good, question, obviously...",[yeah john really good question obviously thou...,45
3,4,1Q23,My first question is you mentioned that your r...,Erika Najarian,"Analyst, UBS Securities LLC","Yeah. So, Erika, as you know, we take \n not g...",Jeremy Barnum,"Chief Financial Officer, JPMorgan Chase & Co.","[first, question, mentioned, reserve, build, d...",[first question mentioned reserve build driven...,"[yeah, erika, know, take, going, lot, detail, ...",[yeah so erika know take going go lot detail t...,17
4,5,1Q23,Hey. Good morning. Maybe just a little bit on ...,Jim Mitchell,"Analyst, Seaport Global Securities LLC","Yeah. A couple things there. So, first of all,...","Jeremy Barnum, Jamie Dimon","Chief Financial Officer, JPMorgan Chase & Co.;...","[hey, good, morning, maybe, little, bit, depos...",[hey good morning maybe little bit deposit tho...,"[yeah, couple, thing, there, first, all, know,...",[yeah couple things there so first all know ri...,31


In [239]:
qa_data = qa_data.drop(columns=["question_tokenised_data", "answer_tokenised_data", "count"])


In [240]:
qa_data.rename(columns={"Quarter-Year":"Quarter", "Asked By":"Analyst", "Role of the person asked the question":"Analyst Role","Answer":"Response", "Answered By": "Executive", "Role of the person answered the question":"Executive Role Type" }, inplace=True)

qa_data.head()

,Index,Quarter,Question,Analyst,Analyst Role,Response,Executive,Executive Role Type,question_cleaned,answer_cleaned
0,1,1Q23,"So, Jamie, I was actually hoping to get your p...",Steven Chubak,"Analyst, Wolfe Research LLC","Well, I think you were already kind of complet...",Jamie Dimon,"Chairman & Chief Executive Officer, JPMorgan C...",[so jamie actually hoping get perspective see ...,[well think already kind complete answering qu...
1,2,1Q23,"Hey, thanks. Good morning. Hey, Jeremy, I was ...",Ken Usdin,"Analyst, Jefferies LLC","Yeah, sure. So let me just summarize the drive...",Jeremy Barnum,"Chief Financial Officer, JPMorgan Chase & Co.",[hey thanks good morning hey jeremy wondering ...,[yeah sure let summarize drivers change outloo...
2,3,1Q23,"Hi, thanks. Jeremy, wanted to follow up again ...",John McDonald,"Analyst, Autonomous Research","Yeah. John, it's a really good question, and w...",Jeremy Barnum,"Chief Financial Officer, JPMorgan Chase & Co.",[hi thanks jeremy wanted follow drivers nii re...,[yeah john really good question obviously thou...
3,4,1Q23,My first question is you mentioned that your r...,Erika Najarian,"Analyst, UBS Securities LLC","Yeah. So, Erika, as you know, we take \n not g...",Jeremy Barnum,"Chief Financial Officer, JPMorgan Chase & Co.",[first question mentioned reserve build driven...,[yeah so erika know take going go lot detail t...
4,5,1Q23,Hey. Good morning. Maybe just a little bit on ...,Jim Mitchell,"Analyst, Seaport Global Securities LLC","Yeah. A couple things there. So, first of all,...","Jeremy Barnum, Jamie Dimon","Chief Financial Officer, JPMorgan Chase & Co.;...",[hey good morning maybe little bit deposit tho...,[yeah couple things there so first all know ri...


In [241]:
print(len(qa_data))

80


In [242]:
# Standardize Executive Roles
for i in range(len(qa_data)):
  qa_data.loc[i, "Executive Role Type"] = re.sub(r"Chairman & Chief Executive Officer, JPMorgan Chase & Co.","CEO", qa_data.loc[i, "Executive Role Type"])
  qa_data.loc[i, "Executive Role Type"] = re.sub(r"Chief Executive Officer, JPMorgan Chase & Co.","CEO", qa_data.loc[i, "Executive Role Type"])
  qa_data.loc[i, "Executive Role Type"] = re.sub(r"Vice Chairman, JPMorgan Chase & Co.","Vice President", qa_data.loc[i, "Executive Role Type"])
  qa_data.loc[i, "Executive Role Type"] = re.sub(r"Managing Director$","Managing Director", qa_data.loc[i, "Executive Role Type"])
  qa_data.loc[i, "Executive Role Type"] = re.sub(r"Head of Investor Relations, JPMorgan Chase & Co.","Head of IR", qa_data.loc[i, "Executive Role Type"])
  qa_data.loc[i, "Executive Role Type"] = re.sub(r"Chief Financial Officer, JPMorgan Chase & Co.","CFO", qa_data.loc[i, "Executive Role Type"])
  qa_data.loc[i, "Executive Role Type"] = re.sub(r"Chief Operating Officer, JPMorgan Chase & Co.","COO", qa_data.loc[i, "Executive Role Type"])
  qa_data.loc[i, "Executive Role Type"] = re.sub(r"Chief Financial Officer, JPMorganChase","CFO", qa_data.loc[i, "Executive Role Type"])
  qa_data.loc[i, "Executive Role Type"] = re.sub(r"Chairman & Chief Executive Officer, JPMorganChase","CEO", qa_data.loc[i, "Executive Role Type"])


In [243]:
#split column Analyst Roles into Analyst and organisation
qa_data["Organisation"] = qa_data["Analyst Role"].apply(lambda x: x.split(",")[1])
qa_data["Analyst Role"] = qa_data["Analyst Role"].apply(lambda x: x.split(",")[0])

In [244]:
col=qa_data.pop("Organisation")
qa_data.insert(5, col.name, col)

In [245]:
qa_data.tail(15)

,Index,Quarter,Question,Analyst,Analyst Role,Organisation,Response,Executive,Executive Role Type,question_cleaned,answer_cleaned
65,73,3Q24,"So Jeremy, how are you? So I do want to ask on...",Steven Chubak,Analyst,Wolfe Research LLC,Sure. So good question and I agree with your n...,Jeremy Barnum,CFO,[jeremy you want ask expenses light comments d...,[sure good question agree numbers agree way no...
66,74,3Q24,"My first question, and thank you very much for...",Erika Najarian,Analyst,UBS Securities LLC,Right. Okay. You threw a curve ball at the end...,Jeremy Barnum,CFO,[first question thank much answering nii quest...,[right okay threw curve ball end there erika e...
67,75,3Q24,So we've seen a couple more banks entering par...,Glenn Schorr,Analyst,Evercore ISI,"Yeah. So let me take time to cover this one, b...",Jamie Dimon,CEO,[seen couple banks entering partnerships alter...,[yeah let take time cover one obviously become...
68,76,3Q24,Can you guys give us some color on the new Bas...,Gerard Cassidy,Analyst,RBC Capital Markets LLC,Yeah. When I said we're at $30 billion of exce...,"Jamie Dimon, Jeremy Barnum","CEO, CFO",[guys give us color new basel iii know specifi...,[yeah said $30 billion excess assuming barrs s...
69,77,3Q24,Good morning. So lower rates was supposed to d...,Matt O'Connor,Analyst,Deutsche Bank Securities,"I would say, Matt, generally no, frankly, with...",Jeremy Barnum,CFO,[good morning lower rates supposed drive pick-...,[would say matt generally no frankly couple mi...
70,79,3Q24,Is the disruption from market makers like Jane...,Ebrahim H. Poonawala,Analyst,Bank of America Merrill Lynch,"Yeah, Ebrahim, the way I would frame that is n...",Jeremy Barnum,CFO,[disruption market makers like jane street fix...,[yeah ebrahim way would frame risk reality alw...
71,81,3Q24,I'm not going to ask about a specific NII in 2...,Saul Martinez,Analyst,HSBC Securities (USA),"Sure. Yeah, thanks for the question, Saul. And...",Jeremy Barnum,CFO,[going ask specific nii 2025 want delve think ...,[sure yeah thanks question saul think laid bui...
72,82,4Q24,"I wanted to ask about capital, and I know you ...",John McDonald,Analyst,Truist Securities,"So, yeah, you've noted all the points that we ...",Jeremy Barnum,CFO,[wanted ask capital know get question lot kind...,[so yeah noted points always make repeat them ...
73,83,4Q24,"Hi. Simple and then more difficult, I guess. J...",Mike Mayo,Analyst,Wells Fargo Securities LLC,I do love what I do. And answering the second ...,Jamie Dimon,CEO,[hi simple difficult guess jamie whos successo...,[love do answering second question first look ...
74,84,4Q24,"Hey. Good morning. Maybe just on regulation, w...",Jim Mitchell,Analyst,Seaport Global Securities LLC,"Hey, Jim. I mean, it's obviously something we'...",Jeremy Barnum,CFO,[hey good morning maybe regulation new adminis...,[hey jim mean obviously something thinking lot...


In [246]:
qa_data.isnull().sum()

,0
Index,0
Quarter,0
Question,0
Analyst,0
Analyst Role,0
Organisation,0
Response,0
Executive,0
Executive Role Type,0
question_cleaned,0


JP Morgan Management Discussion

In [18]:
#defining santader dataframe
jpmorgan_body_df=pd.read_excel("/content/drive/MyDrive/bank_of_england/data/preprocessed_data/JP Mogran processed thru OpenAI/JPMorgan_Management_Discussion_processed_data.xlsx")
jpmorgan_body_df.head()

,Index,Quarter-Year,Text
0,NaN,4Q24,MANAGEMENT DISCUSSION SECTION \n \nOperator : ...
1,NaN,3Q24,MANAGEMENT DISCUSSION SECTION \n \n...
2,NaN,2Q24,MANAGEMENT DISCUSSION SECTION \n \n...
3,NaN,1Q24,MANAGEMENT DISCUSSION SECTION \n \n...
4,NaN,4Q23,MANAGEMENT DISCUSSION SECTION \n \n...


In [19]:
jpmorgan_body_df.drop(columns=["Index"], inplace=True)
jpmorgan_body_df.head()

,Quarter-Year,Text
0,4Q24,MANAGEMENT DISCUSSION SECTION \n \nOperator : ...
1,3Q24,MANAGEMENT DISCUSSION SECTION \n \n...
2,2Q24,MANAGEMENT DISCUSSION SECTION \n \n...
3,1Q24,MANAGEMENT DISCUSSION SECTION \n \n...
4,4Q23,MANAGEMENT DISCUSSION SECTION \n \n...


In [20]:
preprocessor(jpmorgan_body_df, "Text", "text_tokenised_data", "text_cleaned")

In [21]:
jpmorgan_body_df.head()


,Quarter-Year,Text,text_tokenised_data,text_cleaned
0,4Q24,MANAGEMENT DISCUSSION SECTION \n \nOperator : ...,"[management, discussion, section, operator, go...",[management discussion section operator : good...
1,3Q24,MANAGEMENT DISCUSSION SECTION \n \n...,"[management, discussion, section, operator, go...",[management discussion section operator : good...
2,2Q24,MANAGEMENT DISCUSSION SECTION \n \n...,"[management, discussion, section, operator, go...",[management discussion section operator : good...
3,1Q24,MANAGEMENT DISCUSSION SECTION \n \n...,"[management, discussion, section, operator, go...",[management discussion section operator : good...
4,4Q23,MANAGEMENT DISCUSSION SECTION \n \n...,"[management, discussion, section, operator, go...",[management discussion section operator : good...


In [22]:
jpmorgan_body_df.isnull().sum()

,0
Quarter-Year,0
Text,0
text_tokenised_data,0
text_cleaned,0


# **Export the output as a csv file**

JP morgan QA section

In [247]:
#export preprocessed data
preprocessed_qa_csv_path1 = "/content/drive/MyDrive/bank_of_england/data/preprocessed_data/jpmorgan_qna_df_preprocessed_ver9.csv"
qa_data.to_csv(preprocessed_qa_csv_path1, index=False)


In [217]:
print(len(qa_data))

80


JP morgan management discussion

In [24]:
#export preprocessed data
preprocessed_qa_csv_path2 = "/content/drive/MyDrive/bank_of_england/data/preprocessed_data/jpmorgan_management_df_preprocessed_ver8.csv"
jpmorgan_body_df.to_csv(preprocessed_qa_csv_path2, index=False)